<left>FINM 33150 - Quantitative Trading Strategies</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 5: FX Carry Strategy </center></h1>
<center>Due - 23:00 [CST] February 11th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h5> Imports </h5>

<h5> Constants </h5>

<h5> Helper Functions </h5>

<h2> 2. Data </h2>

<h3> 2-1. Raw Data </h3>

<h3> 2-2. Data Processing </h3>

<h3> 2-3. Data Description </h3>

<h2> 3. Analysis </h2>